In [1]:
import base64
import based58
import sys
import json
import os
import time
from pathlib import Path

import solana.rpc.api
from solders.signature import Signature
from solana.exceptions import SolanaRpcException
from solders.rpc.responses import GetTransactionResp

from src.parser import MarginfiTransactionParser


%load_ext autoreload
%autoreload 2
sys.path.append('..')


ppk = "MFv2hWf31Z9kbCa1snEPYctwafyhdvnV7FZnsebVacA"
idl_path = '../marginfi_idl2.json'


def get_tx(tx_signature):
    try:
        transaction = solana_client.get_transaction(
            tx_signature.signature, 
#             Signature.from_string(
#                 "4if5hzfKbJSxbA13ReBca2sY6kbAias7FseHG1QdUCHqSjhG4eNbGoz96TbBSNmCGoSQfKWx5gMtDVgxNb1iUV2M"
#             ),
            'jsonParsed', 
            max_supported_transaction_version=0
        )
        return transaction
    except SolanaRpcException as e:
        print(f"SolanaRpcException while fetching {str(tx_signature.signature)}")
        time.sleep(0.5)
        return get_tx(tx_signature)

    
def tx_printout(tx_parsed):

    
    print('===========logs============')
    for log, value in tx_parsed['logs'].items():
        print(f"{log} \n")
        print(f'      Parsed: {value} \n')
    
    print('=========relevant instructions=========')
    for program_id, instruction_data in tx_parsed['instructons']:
        print(f"Program ID: {program_id} \n   {instruction_data}")
    
    print('============Account Balances==========')
    for i in tx_parsed['account_balances']:
        print(i)
        
    print('============Token Balances==========')
    for i in tx_parsed['token_balances']:
        print(f"{i} \n")       
        

# Parse single TX

In [2]:
solana_client = solana.rpc.api.Client(os.getenv("RPC_URL"))

tx_decoder = MarginfiTransactionParser(
    idl_path,  # mf ild
    solana.rpc.api.Pubkey.from_string(ppk)  # margin fi
)

transaction = solana_client.get_transaction(
    Signature.from_string(
        '64dt9xEoEkrS5ML3Bo5xhpLpJLz7L1BoDJPwTK5XFwbzswtYusiTSgHv1d2rH2o8ao2UzJgVpjgxTHhTuV834iB'
    ), 
    'jsonParsed', 
    max_supported_transaction_version=0
)


In [3]:
## Parser set to printing decoded tx to console (for debug perposes)
# print(tx_decoder._processor)

tx_decoder.decode_tx(transaction.value.transaction)

<ParsedTransactions(
   id=None, 
   transaction_id='64dt9xEoEkrS5ML3Bo5xhpLpJLz7L1BoDJPwTK5XFwbzswtYusiTSgHv1d2rH2o8ao2UzJgVpjgxTHhTuV834iB',

   instruction_name='lending_account_liquidate', 
   event_name='LendingAccountLiquidateEvent',
 
   position='asset', 
   token='EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
   amount=15232047, 
   amount_decimal=6, 
   account='73iDDwiYTz1PTzyWN23w3r3rb2hMUffkZtkVJWVJ9W4P', 
   signer='BXHWiKgXouLRRjKaQ1fQrL1Lb2nuqqYM82tFfAzfNGFs', 
   created_at=None,
   lending_account_id=None)>
<ParsedTransactions(
   id=None, 
   transaction_id='64dt9xEoEkrS5ML3Bo5xhpLpJLz7L1BoDJPwTK5XFwbzswtYusiTSgHv1d2rH2o8ao2UzJgVpjgxTHhTuV834iB',

   instruction_name='lending_account_liquidate', 
   event_name='LendingAccountLiquidateEvent',
 
   position='liability', 
   token='EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
   amount=-15232047, 
   amount_decimal=6, 
   account='3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1', 
   signer='DAhb626CWFq7GwmPPexXBU3jXLHf

## Parse transactions from oldest to newest
- limited to one signer/ one mf account.  - '3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1'


In [4]:
# collect signatures related to marginfi account:
mf_account = solana.rpc.api.Pubkey.from_string("3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1")
mfacc_tx_sinatures = solana_client.get_signatures_for_address(
    mf_account,
    limit = 1000
)
print(f"Transactions to collect: {len(mfacc_tx_sinatures.value)}")
# collect mf account related txs
mfacc_txs = list()

for idx, tx in enumerate(mfacc_tx_sinatures.value):
#     print(idx+1)
    signature = tx.signature
    transaction = get_tx(tx)
    mfacc_txs.append(transaction)
print(f"Collected: {len(mfacc_txs)}")

Transactions to collect: 458


KeyboardInterrupt: 

In [5]:
def dump_json_to_file(json_data, file_path):
    with open(file_path, 'w') as file:
        json.dump(json_data, file, indent=4)

def load_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


# path = './mf_txs/'
# for num, tx in enumerate(mfacc_txs):
#     dump_json_to_file(tx.to_json(), os.path.join(path, f"{'{:03d}'.format(num)}.json"))

#### GET data ffrom local storage:

In [6]:
# mfacc_txs = []
# for file_name in sorted(os.listdir('./mf_txs/')):
#     tx_json = load_json_from_file(os.path.join('./mf_txs/', file_name))
#     tx = GetTransactionResp.from_json(tx_json)
#     mfacc_txs.append(tx)
# mfacc_txs.reverse()
# print(len(mfacc_txs))

429


In [8]:
for tx in mfacc_txs:
    print(f"=====================  Transaction: {tx.value.transaction.transaction.signatures[0]} \n")
    tx_decoder.decode_tx(tx.value.transaction)
    
# print(f"Events collected: {len(tx_decoder.events)}") 

=====================  Transaction: 2nSZ3J63grj5qisbFtCcDr6K46hLmzcL4WtVKVYLYofDT2C6QK1j69AZxChCUs8EDDvzbZhuMDzBjY7fdLgkUSoG 

<LendingAccounts(
   id=None, 
   authority='DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc', 
   address='3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1', 
   group='4qp6Fx6tnZkY5Wropq9wUYgtFxXKwE6viZxFHg3rdAG8',
   created_at=None)>
=====================  Transaction: 2WMjce2dMe2Lm7AXpQaAMrT9qtEfveTeNiSFZqXRkELmT8QhVGcmu8ir7VHcd63BBdaW9GUgqyBCJHubfNNWkynL 

<ParsedTransactions(
   id=None, 
   transaction_id='2WMjce2dMe2Lm7AXpQaAMrT9qtEfveTeNiSFZqXRkELmT8QhVGcmu8ir7VHcd63BBdaW9GUgqyBCJHubfNNWkynL',

   instruction_name='lending_account_deposit', 
   event_name='LendingAccountDepositEvent',
 
   position='asset', 
   token='EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
   amount=1005668538, 
   amount_decimal=None, 
   account='3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1', 
   signer='DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc', 
   created_at=None,
   lending

get data for margin fi account: